<a href="https://colab.research.google.com/github/pdelfidali/SNDL/blob/main/HomeAssignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Home assignment 2
## Piotr del Fidali

### Load MNIST dataset
We load MNIST dataset using function from a keras library. By default, it divides data to 60.000 record for train and 10.000 test sets. We also change the types to float and divide them by 255, so the range of values is (0, 1) rather than (0, 255). Next, we reshape data so the single observation is vector with length of 784 (not matrix with 28x28 size).

In [ ]:
from keras.src.utils import to_categorical
from keras.datasets.mnist import load_data

(x_train, y_train), (x_test, y_test) = load_data()
x_train = x_train.astype('float32') /255.
x_test = x_test.astype('float32') /255.

x_train = x_train.reshape(60000, 28*28)
x_test = x_test.reshape(10000, 28*28)

print("Training data shape:", x_train.shape)
print("Testing data shape:", x_test.shape)

11490434/11490434 [==============================] - 0s 0us/step
Training data shape: (60000, 784)
Testing data shape: (10000, 784)


We define the function that will select only observations for given digits. It will also ensure that the length of y vector is the same as the number of digits in given range. To work properly, argument digit_range should be next digits (ex. [1, 2, 3] but not [2, 5, 7]).

In [ ]:
import numpy as np

def vertical_slice_data(x_data, y_data, digit_range):
    indices = np.where(np.isin(y_data, digit_range))[0]
    y_sliced = y_data[indices]
    y_sliced = y_sliced - min(digit_range)
    return x_data[indices], to_categorical(y_sliced)

x_train_0_to_2, y_train_0_to_2 = vertical_slice_data(x_train, y_train, [0, 1, 2])
x_train_3_to_5, y_train_3_to_5 = vertical_slice_data(x_train, y_train, [3, 4, 5])
x_train_6_to_9, y_train_6_to_9 = vertical_slice_data(x_train, y_train, [6, 7, 8, 9])

x_test_0_to_2, y_test_0_to_2 = vertical_slice_data(x_test, y_test, [0, 1, 2])
x_test_3_to_5, y_test_3_to_5 = vertical_slice_data(x_test, y_test, [3, 4, 5])
x_test_6_to_9, y_test_6_to_9 = vertical_slice_data(x_test, y_test, [6, 7, 8, 9])


print("Amount of samples in training datasets after slicing:")
print(f'0-2: {y_train_0_to_2.shape} - {len(y_train_0_to_2)/len(y_train):.2%} of full training')
print(f'3-5: {y_train_3_to_5.shape} - {len(y_train_3_to_5)/len(y_train):.2%} of full training')
print(f'6-9: {y_train_6_to_9.shape} - {len(y_train_6_to_9)/len(y_train):.2%} of full training')

print("Amount of samples in test datasets after slicing:")
print(f'0-2: {y_test_0_to_2.shape} - {len(y_test_0_to_2)/len(y_test):.2%} of full test')
print(f'3-5: {y_test_3_to_5.shape} - {len(y_test_3_to_5)/len(y_test):.2%} of full test')
print(f'6-9: {y_test_6_to_9.shape} - {len(y_test_6_to_9)/len(y_test):.2%} of full test')

Amount of samples in training datasets after slicing:
0-2: (18623, 3) - 31.04% of full training
3-5: (17394, 3) - 28.99% of full training
6-9: (23983, 4) - 39.97% of full training
Amount of samples in test datasets after slicing:
0-2: (3147, 3) - 31.47% of full test
3-5: (2884, 3) - 28.84% of full test
6-9: (3969, 4) - 39.69% of full test


## Train local models using each remote local dataset

We train three models for 0–2, 3–5, 6–9 digits. We  

In [ ]:
from keras.models import Sequential
from keras.layers import Input, Dense
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, recall_score, precision_score, accuracy_score

model_0_to_2 = Sequential()
model_0_to_2.add(Dense(512, activation='relu', input_shape=(28*28, )))
model_0_to_2.add(Dense(3, activation='softmax'))
model_0_to_2.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model_0_to_2.summary()

model_0_to_2.fit(x_train_0_to_2, y_train_0_to_2, epochs=10, batch_size=128)
test_loss, test_accu = model_0_to_2.evaluate(x_test_0_to_2, y_test_0_to_2)
y_pred_0_to_2 = model_0_to_2.predict(x_test_0_to_2)
test_recall = recall_score(y_test_0_to_2.argmax(1), y_pred_0_to_2.argmax(1), average=None)
test_precision = precision_score(y_test_0_to_2.argmax(1), y_pred_0_to_2.argmax(1), average=None)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 3)                 1539      
                                                                 
Total params: 403459 (1.54 MB)
Trainable params: 403459 (1.54 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
146/146 [==============================] - 2s 7ms/step - loss: 0.0645 - accuracy: 0.9797
Epoch 2/10
146/146 [==============================] - 1s 5ms/step - loss: 0.0214 - accuracy: 0.9935
Epoch 3/10
146/146 [==============================] - 1s 6ms/step - loss: 0.0129 - accuracy: 0.9960
Epoch 4/10
146/146 [==============================] - 1s 6ms/step - loss: 0.0089 - accuracy: 0.9974
Epoch 5/10
146/

In [ ]:
print('Model for 0, 1 and 2 digits classification')
print(f'\tTest loss: {test_loss:5f}\n\tTest accuracy: {test_accu:5f}')
for class_idx, recall in enumerate(test_recall):
    print(f"\tRecall for Class {class_idx}: {recall:.5f}")
for class_idx, recall in enumerate(test_precision):
    print(f"\tPrecision for Class {class_idx}: {recall:.5f}")

In [ ]:
cm_0_to_2 = confusion_matrix(y_test_0_to_2.argmax(1), y_pred_0_to_2.argmax(1))
disp = ConfusionMatrixDisplay(cm_0_to_2, display_labels=[0, 1, 2])
disp.plot(cmap='winter')

In [ ]:
model_3_to_5 = Sequential()
model_3_to_5.add(Dense(512, activation='relu', input_shape=(28*28, )))
model_3_to_5.add(Dense(3, activation='softmax'))
model_3_to_5.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model_3_to_5.summary()

model_3_to_5.fit(x_train_3_to_5, y_train_3_to_5, epochs=10, batch_size=128)
test_loss, test_accu = model_3_to_5.evaluate(x_test_3_to_5, y_test_3_to_5)
y_pred_3_to_5 = model_3_to_5.predict(x_test_3_to_5)
test_recall = recall_score(y_test_3_to_5.argmax(1), y_pred_3_to_5.argmax(1), average=None)
test_precision = precision_score(y_test_3_to_5.argmax(1), y_pred_3_to_5.argmax(1), average=None)

In [ ]:
print('Model for 3, 4 and 5 digits classification')
print(f'\tTest loss: {test_loss:5f} \n\tAccuracy: {test_accu:.5f}')
for class_idx, recall in enumerate(test_recall):
    print(f"\tRecall for Class {class_idx + 3}: {recall:.5f}")
for class_idx, recall in enumerate(test_precision):
    print(f"\tPrecision for Class {class_idx + 3}: {recall:.5f}")

In [ ]:
cm_3_to_5 = confusion_matrix(y_test_3_to_5.argmax(1), y_pred_3_to_5.argmax(1))
disp = ConfusionMatrixDisplay(cm_3_to_5, display_labels=[3, 4, 5])
disp.plot(cmap='winter')

In [ ]:
model_6_to_9 = Sequential()
model_6_to_9.add(Dense(512, activation='relu', input_shape=(28*28, )))
model_6_to_9.add(Dense(4, activation='softmax'))
model_6_to_9.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model_6_to_9.summary()


model_6_to_9.fit(x_train_6_to_9, y_train_6_to_9, epochs=10, batch_size=128)
test_loss, test_accu = model_6_to_9.evaluate(x_test_6_to_9, y_test_6_to_9)
y_pred_6_to_9 = model_6_to_9.predict(x_test_6_to_9)
test_recall = recall_score(y_test_6_to_9.argmax(1), y_pred_6_to_9.argmax(1), average=None)
test_precision = precision_score(y_test_6_to_9.argmax(1), y_pred_6_to_9.argmax(1), average=None)

In [ ]:
print('Model for 6, 7, 8 and 9 digits classification')
print(f'\tTest loss: {test_loss:5f} \n\tAccuracy: {test_accu:.5f}')
for class_idx, recall in enumerate(test_recall):
    print(f"\tRecall for Class {class_idx + 6}: {recall:.5f}")
for class_idx, recall in enumerate(test_precision):
    print(f"\tPrecision for Class {class_idx + 6}: {recall:.5f}")

In [ ]:
cm_6_to_9 = confusion_matrix(y_test_6_to_9.argmax(1), y_pred_6_to_9.argmax(1))
disp = ConfusionMatrixDisplay(cm_6_to_9, display_labels=[6, 7, 8, 9])
disp.plot(cmap='winter')

### Aggregate the local models into a global model

We create an Input layer that will take a single image and will pass it to each of our defined models. Then we combine outputs of models to a single vector of length 10, because of making models take next digits, we do not have to worry about any mapping.

In [ ]:
from keras.models import Model
from keras.layers import Input, concatenate

common_input = Input(shape=(784,))

output_model_1 = model_0_to_2(common_input)
output_model_2 = model_3_to_5(common_input)
output_model_3 = model_6_to_9(common_input)

combined_output = concatenate([output_model_1, output_model_2, output_model_3])

combined_model = Model(inputs=common_input, outputs=combined_output)

In [ ]:
y_pred = combined_model.predict(x_test)
test_accu = accuracy_score(y_test, y_pred.argmax(1))
test_recall = recall_score(y_test, y_pred.argmax(1), average=None)
test_precision = precision_score(y_test, y_pred.argmax(1), average=None)

In [ ]:
print('Combined model digits classification')
print(f'\tAccuracy: {test_accu:.5f}')
for class_idx, recall in enumerate(test_recall):
    print(f"\tRecall for Class {class_idx}: {recall:.5f}")
for class_idx, recall in enumerate(test_precision):
    print(f"\tPrecision for Class {class_idx}: {recall:.5f}")

In [ ]:
cm_combined_model = confusion_matrix(to_categorical(y_test).argmax(1), y_pred.argmax(1))
disp = ConfusionMatrixDisplay(cm_combined_model)
disp.plot(cmap='winter')

The combined model is not performing as well as models that predict only three or four classes. Although its performance is not as good, we can see that it does not make miss classifications inside the group it was learned in. For example, for digit range 0–2 there are only two miss classifications as another digit from the same range. For the second range it is also two and for third it's 3.  There is a lot '2' misclassified as '3' we can see that in confusion matrix. Recall value is smallest for class '2,' precision is worst for both class '2' and '3.' Overall, the model is performing well.

## Model trained on the full dataset

In [ ]:
y_train = to_categorical(y_train)

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(28*28, )))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

model.fit(x_train, y_train, epochs=10, batch_size=128)
y_pred = model.predict(x_test)
test_accu = accuracy_score(y_test, y_pred.argmax(1))
test_recall = recall_score(y_test, y_pred.argmax(1), average=None)
test_precision = precision_score(y_test, y_pred.argmax(1), average=None)

In [ ]:
print('Model trained for full dataset digits classification')
print(f'\tAccuracy: {test_accu:.5f}')
for class_idx, recall in enumerate(test_recall):
    print(f"\tRecall for Class {class_idx}: {recall:.5f}")
for class_idx, recall in enumerate(test_precision):
    print(f"\tPrecision for Class {class_idx}: {recall:.5f}")

In [ ]:
cm_full_model = confusion_matrix(y_test, y_pred.argmax(1))
disp = ConfusionMatrixDisplay(cm_full_model)
disp.plot(cmap='winter')

The model trained on full dataset, performance is much better than the combined model.

## Conclusion
Training the models for subsets of data might be beneficial if the dataset is huge:
   - it can save time for training
   - it can help to remove some classification errors, as we could observe there was little number of errors inside a specific group.


The model performance (all metrics) is better if we train the model on full dataset.